In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus

In [2]:
client = QdrantClient(host="localhost", port=6333)
client

In [3]:
import numpy as np

# Load embeddings
embeddings = np.load('embeddings_combined1.npy')

In [4]:
import numpy as np

# Assuming you have already loaded your embeddings from 'embeddings.npy'
embeddings = np.load('embeddings_combined1.npy')

# Convert the data type of embeddings to float64
embeddings_float64 = embeddings.astype(np.float64)

# Optionally, you can save the converted embeddings back to a file
np.save('embeddings_float641.npy', embeddings_float64)

# To verify, you can check the dtype and values of the first few embeddings
print("Data type:", embeddings_float64.dtype)
print("First two embeddings:", embeddings_float64[:2])


Data type: float64
First two embeddings: [[-0.36123669  0.6647532  -0.17255768 ... -0.56919503 -0.24727423
   0.08984298]
 [-0.00590212  1.42034149  0.1689119  ... -0.07028337 -0.10406235
   0.04801601]]


In [5]:
embeddings = np.load('embeddings_float641.npy')
print(len(embeddings[1]))
#embeddings[1]

768


In [6]:
index = list(range(len(embeddings)))
index[-10:]

[27543, 27544, 27545, 27546, 27547, 27548, 27549, 27550, 27551, 27552]

In [7]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import CollectionConfig, Distance, VectorParams

client = QdrantClient(host='localhost', port=6333)

client.recreate_collection(
    collection_name="combined_collection1",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

True

In [8]:
my_collection = "combined_collection1"
batch_size = 1000  # You can adjust this size based on your system's capability
num_batches = len(embeddings) // batch_size + (1 if len(embeddings) % batch_size else 0)

for batch_num in range(num_batches):
    start_index = batch_num * batch_size
    end_index = min(start_index + batch_size, len(embeddings))
    batch_embeddings = embeddings[start_index:end_index]

    # Create a list of IDs for the current batch
    batch_ids = list(range(start_index, end_index))

    # Upload the batch
    client.upsert(
        collection_name=my_collection,
        points=models.Batch(
            ids=batch_ids,
            vectors=batch_embeddings.tolist()
        )
    )

In [9]:
# Fetch the record with ID 100
response = client.retrieve(
    collection_name=my_collection,
    ids=[768],
    with_vectors=True  # Retrieve the vectors as well
)

# Extract records from the response
records = response

# Print the details of the retrieved record
for record in records:
    print(f"ID: {record.id}, Vector: {record.vector}")

ID: 768, Vector: [-0.06356667, 0.03297227, -0.002508327, -0.008032285, 0.0024353561, -0.08241756, 0.041522555, -0.02142113, 0.03195651, -0.012070604, 0.045497663, 0.00843973, 0.034037717, 0.036957674, -0.007306505, 0.048293166, -0.016281737, -0.0054817568, 0.022272287, -0.014114783, -0.009133766, -0.07734295, 0.046993423, 0.0052702017, 0.06294905, 0.061794065, -0.018073754, 0.043974087, -0.011241537, 0.01613725, -0.003028938, 0.012623817, -0.005756398, -0.019823324, 0.038137287, 0.08299642, -0.0100848405, 0.031896304, 0.00069233874, -0.04941294, -0.0055652424, -0.053654518, 0.054549124, -0.020243853, -0.05719094, -0.04315307, -0.044521876, 0.028169166, 0.03911142, -0.047430903, 0.023562033, -0.009110954, 0.06971925, -0.019571858, -0.046363935, 0.04604607, -0.032245338, -0.01486379, -0.029707693, -0.047219984, -0.04880291, 0.004384091, -0.02645209, 0.019878317, 0.0037926573, 0.011795709, 0.02326741, 0.0124156345, -0.02681511, -0.0019711957, -0.018403461, -0.033118125, -0.0042509437, -0.

In [10]:
pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

# Initialize the Sentence Transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Connect to Qdrant
client = QdrantClient(host="localhost", port=6333)  # Replace with your Qdrant host and port

# Query sentence
query_sentence = "Vanilla Gold Ice Cream"
query_embedding = model.encode(query_sentence)

# Perform the search
search_results = client.search(
    collection_name=my_collection,
    query_vector=query_embedding.tolist(),
    query_filter=None,  # You can add filters if needed
    limit=10  # Number of top similar results
)

# Output the results
# Adjust the way you access the results based on the actual structure of search_results
# Here's an example assuming search_results is a list
for hit in search_results:
    print(f"ID: {hit.id}, Score: {hit.score}")


ID: 13872, Score: 0.85485965
ID: 25751, Score: 0.85057247
ID: 27497, Score: 0.8378357
ID: 11763, Score: 0.8321847
ID: 8085, Score: 0.83204174
ID: 11828, Score: 0.8318955
ID: 2871, Score: 0.83033824
ID: 16701, Score: 0.82868177
ID: 9915, Score: 0.82634723
ID: 18467, Score: 0.82564026


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

# Load the CSV file
df = pd.read_csv('bigBasketProducts.csv')

# Initialize the Sentence Transformer and Qdrant Client
model = SentenceTransformer('bert-base-nli-mean-tokens')
client = QdrantClient(host="localhost", port=6333)

# Initialize the T5 model and tokenizer
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Function to generate a response based on enriched product information using T5
def generate_response(product_info):
    enriched_description = f"summarize: {product_info['brand']} {product_info['type']} in {product_info['category']}: {product_info['description']}"
    input_ids = t5_tokenizer.encode(enriched_description, return_tensors='pt')
    output_ids = t5_model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    return t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Query the Qdrant database
query_sentence = "drinks for soft skin "
query_embedding = model.encode(query_sentence)
search_results = client.search(collection_name=my_collection, query_vector=query_embedding.tolist(), limit=5)

# Process the search results and generate responses
for hit in search_results:
    # Retrieve the enriched product information using the hit ID
    product_info = df.loc[df['index'] == hit.id].iloc[0]
    contextual_response = generate_response(product_info)
    print(f"ID: {hit.id}, Response: {contextual_response}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
